In [ ]:
import tensorflow as tf

import os
import pathlib
import time
import datetime

from matplotlib import pyplot

In [ ]:
%%capture
!wget https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz -O facades.tar.gz
!tar -zxvf facades.tar.gz -C ./

In [ ]:
print("학습 데이터셋 A와 B의 개수:", len(next(os.walk('./facades/train/'))[2]))
print("평가 데이터셋 A와 B의 개수:", len(next(os.walk('./facades/val/'))[2]))
print("테스트 데이터셋 A와 B의 개수:", len(next(os.walk('./facades/test/'))[2]))

In [ ]:
# 한 쌍의 이미지 출력(왼쪽은 정답 이미지, 오른쪽은 조건 이미지)
image = Image.open('./facades/train/1.jpg')
print("이미지 크기:", image.size)

plt.imshow(image)
plt.show()

In [ ]:
def load_image_train(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = random_jitter(input_image, real_image)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

In [ ]:
def UNetDown(in_channels, out_channels, normalize=True, dropout=0.0) :
    initializer = tf.random_normal_initializer(0.0, 0.02)

    layers = [ tf.keras.layers.Conv2D(filters = out_channels, kernel_size = 4, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False) ]
    if normalize :
        layers.append( tf.keras.layers.BatchNormalization(axis=[0,1]) )
    if dropout :
        layers.append( tf.keras.layers.Dropout(dropout) )
    layers.append(tf.keras.layers.LeakyReLU(0.2))
    model = tf.keras.Sequential( layers )
    return model

def UNetUp(in_channels, out_channels, dropout=0.0) :
    initializer = tf.random_normal_initializer(0.0, 0.02)
    layers = [ tf.keras.layers.Conv2DTranspose(filters = out_channels, kernel_size = 4, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False) ]
    layers.append( tf.keras.layers.BatchNormalization(axis=[0,1]) )
    layers.append( tf.keras.layers.ReLU() )
    if dropout :
        layers.append( tf.keras.layers.Dropout(dropout) )
    model = tf.keras.Sequential(layers)
    return model

def GeneratorUNet(in_channels = 3, out_channels = 3):
    inputs = tf.keras.layers.Input(shape=[256, 256, in_channels])
    downs = [
        UNetDown(3, 64, normalize=False),
        UNetDown(64, 128),
        UNetDown(128, 256),
        UNetDown(256, 512, dropout = 0.5),
        UNetDown(512, 512, dropout = 0.5),
        UNetDown(512, 512, dropout = 0.5),
        UNetDown(512, 512, dropout = 0.5),
        UNetDown(512, 512, normalize = False, dropout = 0.5),
    ]
    ups = [
        UNetUp(512, 512, dropout=0.5),
        UNetUp(1024, 512, dropout=0.5),
        UNetUp(1024, 512, dropout=0.5),  
        UNetUp(1024, 512, dropout=0.5),  
        UNetUp(1024, 256),  
        UNetUp(512, 128),  
        UNetUp(256, 64)  
    ]

    final_init = tf.random_normal_initializer(0.0, 0.02)
    final = tf.keras.layers.Conv2DTranspose(out_channels, kennel_size= 4, strides=2,
                                         padding='same',
                                         kernel_initializer=final_init,
                                         activation='tanh')  # (batch_size, 256, 256, 3)


    ### forward   
    x = inputs
    skip_connections = list()
    for i in range(len(downs)) :
        x = downs[i](x)
        if i < len(downs)-1 :
            skip_connections.append(x)

    skip_connections.reverse()

    for i in range(len(ups)) :
        x = ups[i](x)
        x = tf.keras.layers.Concatenate()([x, skip_connections[i]])
    x = final(x)

    return tf.keras.Model(inputs = inputs, outputs = x)

def Discriminator(in_channels = 3) :

    def discriminator_block(in_channels, out_channels, normalization=True):
        initializer = tf.random_normal_initializer(0., 0.02)
        layers = [ tf.keras.layers.Conv2D(filters = out_channels, kernel_size = 4, strides=2, padding='same',
                                    kernel_initializer=initializer, use_bias=False) ]
        if normalization :
            layers.append( tf.keras.layers.BatchNormalization(axis=[0,1]) )
        layers.append(tf.keras.layers.LeakyReLU(0.2))
        return layers
    
    initializer = tf.random_normal_initializer(0., 0.02)
    layers = list()
    layers.extend( discriminator_block(in_channels*2, 64, normalization = False))
    layers.extend( discriminator_block(64, 128) )
    layers.extend( discriminator_block(128, 256) )
    layers.extend( discriminator_block(256, 512) )

    layers.append(tf.keras.layers.ZeroPadding2D(padding = ((1, 0), (1, 0))))
    layers.append(tf.keras.layers.Conv2D(filters = 1, kernel_size = 4, strides=1, padding='same',
                                    kernel_initializer=initializer, use_bias=False))
    model = tf.keras.Sequential(layers)

    ### forward
    img_A = tf.keras.layers.Input(shape=[256, 256, 3], name='input')
    img_B = tf.keras.layers.Input(shape=[256, 256, 3], name='target')
    x = tf.keras.layers.Concatenate()([img_A, img_B])
    x = model(x)

    return tf.keras.Model(inputs = [img_A, img_B], outputs = x)

In [ ]:
criterion_GAN = 